In [1]:
import pandas as pd
import numpy as np

In [2]:
df_pc_toronto = pd.read_csv('Toronto_postal_codes.csv')
df_pc_toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [3]:
df_postal_code1 = df_pc_toronto[df_pc_toronto.Borough != "Not assigned"]
df_postal_code1.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [4]:
condition = df_postal_code1['Neighbourhood'] == 'Not assigned'
df_postal_code1.loc[condition, 'Neighbourhood'] = df_postal_code1['Borough']

df_postal_code1.head()


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [5]:
df_postal_code2 = df_postal_code1.groupby(['Postcode','Borough'], sort = False, as_index=False).agg(lambda x: ','.join(x))
df_postal_code2.head()


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [6]:
df_postal_code2.shape

(103, 3)

In [7]:
df_latlon = pd.read_csv('Geospatial_Coordinates.csv')
df_latlon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
df_final = pd.merge(df_postal_code2, df_latlon, how='outer', left_on='Postcode', right_on='Postal Code')

df_final.drop('Postal Code', axis=1)
df_final.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,M7A,43.662301,-79.389494


In [9]:
df_toronto = df_final[df_final['Borough'].str.contains('Toronto', case= False) ]
df_toronto.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",M5A,43.654260,-79.360636
9,M5B,Downtown Toronto,"Ryerson,Garden District",M5B,43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,M4E,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,M5E,43.644771,-79.373306


In [10]:
##### We will take the 'East Toronto' and explore the points of foursquare point of that area

In [11]:

import requests # library to handle requests
from pandas.io.json import json_normalize
import folium # plotting library


In [12]:
CLIENT_ID = 'WDKVMU4XRDDSFDYUT2EZT5J1HEOY2GZJSED3DERKVXZVZQPB' # your Foursquare ID
CLIENT_SECRET = '0JETOZ2PW3A21GPGVWIC02UKJM3JMOEUNQK0NAL0P4HSO04F' # your Foursquare Secret
VERSION = '20190825'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WDKVMU4XRDDSFDYUT2EZT5J1HEOY2GZJSED3DERKVXZVZQPB
CLIENT_SECRET:0JETOZ2PW3A21GPGVWIC02UKJM3JMOEUNQK0NAL0P4HSO04F


In [13]:
search_query = ''
radius = 500
latitude = 43.676357
longitude = -79.293031

In [14]:
# 1. Search for a specific venue category
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=WDKVMU4XRDDSFDYUT2EZT5J1HEOY2GZJSED3DERKVXZVZQPB&client_secret=0JETOZ2PW3A21GPGVWIC02UKJM3JMOEUNQK0NAL0P4HSO04F&ll=43.676357,-79.293031&v=20190825&query=&radius=500&limit=30'

In [15]:
results = requests.get(url).json()
#results

In [16]:
#We get data from JSON
result = results['response']['venues']

#We transform to Pandas DataFrame
df = json_normalize(result)
df.head()


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d162941735', 'name': 'O...",False,56afcad6498e05333bf42031,NaN,CA,NaN,Canada,NaN,141,[Canada],"[{'label': 'display', 'lat': 43.67629984029563...",43.676300,-79.294784,NaN,NaN,NaN,Glen Stewart Ravine,v-1568127273,NaN
1,"[{'id': '4bf58dd8d48988d159941735', 'name': 'T...",False,4bd461bc77b29c74a07d9282,Glen Manor,CA,Toronto,Canada,Queen St.,89,"[Glen Manor (Queen St.), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.67682094413784...",43.676821,-79.293942,NaN,NaN,ON,Glen Manor Ravine,v-1568127273,NaN
2,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",False,4dbdfa5effcbcd485670ef68,121 Kingston Rd,CA,Toronto,Canada,Dundas St E,1658,"[121 Kingston Rd (Dundas St E), Toronto ON M4L...","[{'label': 'display', 'lat': 43.66925300793846...",43.669253,-79.311140,NaN,M4L 1T2,ON,Beaches Fitness - Personal Trainer & Health Coach,v-1568127273,64837476
3,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4dbc8fe96a23e294ba3237bd,131 Glen Manor Drive,CA,Toronto,Canada,NaN,177,"[131 Glen Manor Drive, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.67527822698259...",43.675278,-79.294647,NaN,NaN,ON,Glen Stewart Park,v-1568127273,NaN
4,[],False,4e58599c887710a848247531,Kingston rd,CA,Toronto,Canada,Main st,449,"[Kingston rd (Main st), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.679822, 'lng':...",43.679822,-79.295899,NaN,NaN,ON,Kondope,v-1568127273,NaN


In [17]:
filtered_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']

type(filtered_columns)
filtered_columns

['name',
 'categories',
 'location.address',
 'location.cc',
 'location.city',
 'location.country',
 'location.crossStreet',
 'location.distance',
 'location.formattedAddress',
 'location.labeledLatLngs',
 'location.lat',
 'location.lng',
 'location.neighborhood',
 'location.postalCode',
 'location.state',
 'id']

In [18]:
df_filtered = df.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


df_filtered['categories'] = df_filtered.apply(get_category_type, axis=1)

df_filtered.columns = [column.split('.')[-1] for column in df_filtered.columns ]
df_filtered.head()


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Glen Stewart Ravine,Other Great Outdoors,NaN,CA,NaN,Canada,NaN,141,[Canada],"[{'label': 'display', 'lat': 43.67629984029563...",43.676300,-79.294784,NaN,NaN,NaN,56afcad6498e05333bf42031
1,Glen Manor Ravine,Trail,Glen Manor,CA,Toronto,Canada,Queen St.,89,"[Glen Manor (Queen St.), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.67682094413784...",43.676821,-79.293942,NaN,NaN,ON,4bd461bc77b29c74a07d9282
2,Beaches Fitness - Personal Trainer & Health Coach,Gym / Fitness Center,121 Kingston Rd,CA,Toronto,Canada,Dundas St E,1658,"[121 Kingston Rd (Dundas St E), Toronto ON M4L...","[{'label': 'display', 'lat': 43.66925300793846...",43.669253,-79.311140,NaN,M4L 1T2,ON,4dbdfa5effcbcd485670ef68
3,Glen Stewart Park,Park,131 Glen Manor Drive,CA,Toronto,Canada,NaN,177,"[131 Glen Manor Drive, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.67527822698259...",43.675278,-79.294647,NaN,NaN,ON,4dbc8fe96a23e294ba3237bd
4,Kondope,None,Kingston rd,CA,Toronto,Canada,Main st,449,"[Kingston rd (Main st), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.679822, 'lng':...",43.679822,-79.295899,NaN,NaN,ON,4e58599c887710a848247531


In [19]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
#folium.features.CircleMarker(
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
#    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(df_filtered.lat, df_filtered.lng, df_filtered.categories):
#    folium.features.CircleMarker(
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map